# Analyse der PV-Vergütungen in der Schweiz
Dieses Notebook analysiert die Photovoltaik-Anlagen Vergütungen in der Schweiz, einschliesslich ihrer Verteilung und Entwicklung.

## Daten laden, aufbereiten und anreichern
In diesem Abschnitt werden die Daten geladen, gefiltert und für die Analyse vorbereitet.

In [1]:
# Bibliotheken importieren
import pandas as pd

# CSV-Dateien laden
evu_df = pd.read_csv('../data/vese/evu_ergebnisse.csv', sep=';')  # EVU-Daten
gemeinde_df = pd.read_csv('../data/vese/gemeinde_ergebnisse.csv', sep=';')  # Gemeinde-Daten

# Datenframes basierend auf `nrElcom` zusammenführen
merged_df = pd.merge(evu_df, gemeinde_df, on='nrElcom', how='inner')  # Inner Join

# Gemeindestand.csv laden, um Kantone und BFS Gde-nummer zuzuordnen
gemeindestand_df = pd.read_csv('../data/vese/Gemeindestand.csv', sep=';')  # Mapping-Daten

# Mapping-Dictionary für Kantone und BFS Gde-nummer erstellen
gemeinde_to_kanton = gemeindestand_df.set_index('BFS Gde-nummer')['Kanton'].to_dict()

# Kantone und BFS Gde-nummer zuordnen
merged_df['Kanton'] = merged_df['idofs'].map(gemeinde_to_kanton)

# Fehlende Werte in 'energy1' und 'eco1' mit 0 auffüllen
merged_df = merged_df.copy()  # Sicherstellen, dass auf einer Kopie gearbeitet wird
merged_df['energy1'] = merged_df['energy1'].fillna(0)  # NaN durch 0 ersetzen
merged_df['eco1'] = merged_df['eco1'].fillna(0)

# Zeilen entfernen, in denen sowohl 'energy1' als auch 'eco1' 0 sind
merged_df = merged_df[~((merged_df['energy1'] == 0) & (merged_df['eco1'] == 0))]

# Vergütung berechnen
merged_df['remuneration'] = merged_df['energy1'] + merged_df['eco1']  # Gesamtvergütung berechnen

## Interaktives Balkendiagramm mit Jahr-Schieberegler erstellen

In [3]:
import plotly.express as px

# Gruppieren nach Jahr & Kanton
df = merged_df.groupby(['year', 'Kanton'])['remuneration'].mean().reset_index()
df.rename(columns={'remuneration': 'avg_remuneration'}, inplace=True)
df['avg_remuneration'] = df['avg_remuneration'].round(2)

# Top 4 Kantone pro Jahr einfärben
top_cantons_per_year = (
    df.groupby('year', group_keys=False)
    .apply(lambda g: g.nlargest(4, 'avg_remuneration')['Kanton'].tolist())
    .to_dict()
)

def assign_color_group(row, top_dict):
    return row['Kanton'] if row['Kanton'] in top_dict.get(row['year'], []) else 'Other'

df['canton_group'] = df.apply(assign_color_group, axis=1, top_dict=top_cantons_per_year)

# Balkendiagramm erstellen
fig = px.bar(
    df,
    x='Kanton',
    y='avg_remuneration',
    animation_frame='year',
    color='canton_group',
    text='avg_remuneration',
    labels={'avg_remuneration': 'Durchschnittliche Vergütung (Rp/kWh)', 'Kanton': 'Kanton'},
    color_discrete_map={'Other': '#D9D9D9'},
    template='simple_white'
)

fig.update_traces(texttemplate='%{text:.0f}', textposition='inside')

fig.update_layout(
    xaxis=dict(categoryorder='total descending', title='Kanton'),
    yaxis=dict(range=[0, 35], title='Durchschnittliche Vergütung (Rp/kWh)'),
    title='Durchschnittliche Einspeisevergütung pro Kanton und Jahr (absteigend sortiert)',
    showlegend=False
)

fig.show()
fig.write_html("../docs/assets/diagramme/pv_verguetungen_sortiert_korrekt.html")


C:\Users\X250\AppData\Local\Temp\ipykernel_6892\2893862314.py:11: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



## Choroplethenkarte von der Vergütung der PV Anlagen

In [ ]:
import folium
import json
from shapely.geometry import shape, mapping
from shapely.ops import transform
from functools import partial
import pyproj

df = merged_df[merged_df['year'] == 2024].copy()

# Prepare cumulative data
cumulative_map_data = df.groupby(['idofs'])['remuneration'].mean().reset_index()

# Load and simplify GeoJSON
with open('../data/electricity_production_plants/gemeinden.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Assign default values to missing municipalities
average_remuneration = cumulative_map_data['remuneration'].mean()
missing_bfs = set(geojson_data['features'][i]['properties']['gemeinde_BFS_NUMMER'] for i in range(len(geojson_data['features']))) - set(cumulative_map_data['idofs'])
for bfs in missing_bfs:
    cumulative_map_data = pd.concat([cumulative_map_data, pd.DataFrame({'idofs': [bfs], 'remuneration': [average_remuneration]})], ignore_index=True)

simplified_features = []
for feature in geojson_data['features']:
    geom = shape(feature['geometry'])
    simplify = partial(
        pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:4326', always_xy=True).transform
    )
    simplified_geom = transform(simplify, geom).simplify(0.001, preserve_topology=True)
    feature['geometry'] = mapping(simplified_geom)
    bfs_nr = feature['properties']['gemeinde_BFS_NUMMER']
    remuneration = cumulative_map_data.loc[cumulative_map_data['idofs'] == bfs_nr, 'remuneration'].values[0] if bfs_nr in cumulative_map_data['idofs'].values else 0
    feature['properties']['remuneration'] = f"{remuneration:.2f}"
    feature['properties']['bfs-str'] = str(feature['properties']['gemeinde_BFS_NUMMER'])
    simplified_features.append(feature)

geojson_data['features'] = simplified_features

map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Add choropleth layer with optimized color palette
folium.Choropleth(
    geo_data=geojson_data,
    name='Choropleth',
    data=cumulative_map_data,
    columns=['idofs', 'remuneration'],
    key_on='feature.properties.gemeinde_BFS_NUMMER',
    fill_color='RdYlGn',  # Reverse color palette for green to red
    fill_opacity=0.9,  # Increased opacity for better visibility
    line_opacity=0.6,  # Slightly thicker boundary lines
    line_color='black',
    legend_name='Vergütung (Rp/kWh)',
    threshold_scale=[0, 5, 10, 15, 20, 25, cumulative_map_data['remuneration'].max()],  # Adjust thresholds to include all values
).add_to(map)

# Add tooltips and customize boundary style
folium.GeoJson(
    geojson_data,
    name="Gemeinden",
    style_function=lambda feature: {
        'color': 'black',
        'weight': 0.5,  # Slightly thicker boundary lines
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['gemeinde_NAME', 'bfs-str', 'remuneration'],
        aliases=['Gemeinde:', 'BFS-Nummer:', 'Vergütung (Rp/kWh):'],
        localize=True
    )
).add_to(map)

# Add title and save map
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Choroplethenkarte von der Vergütung der PV Anlagen pro Gemeinde im Jahr 2024</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

map.save('../docs/assets/diagramme/map_pv_vergütungen.html')

map